In [2]:
!pip install playwright

In [3]:
!pip install beautifulsoup4

In [4]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [5]:
SEASONS = list(range(2010,2024))

In [6]:
SEASONS

[2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

In [7]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [8]:
STANDINGS_DIR

'data/standings'

In [12]:
!playwright install

162.8 MiB [] 0% 0.0s162.8 MiB [] 0% 29.7s162.8 MiB [] 0% 24.6s162.8 MiB [] 0% 13.4s162.8 MiB [] 0% 10.0s162.8 MiB [] 1% 6.0s162.8 MiB [] 1% 5.1s162.8 MiB [] 2% 4.6s162.8 MiB [] 2% 4.5s162.8 MiB [] 3% 4.5s162.8 MiB [] 3% 4.4s162.8 MiB [] 4% 4.2s162.8 MiB [] 4% 4.1s162.8 MiB [] 5% 3.8s162.8 MiB [] 5% 3.7s162.8 MiB [] 6% 3.6s162.8 MiB [] 7% 3.6s162.8 MiB [] 8% 3.4s162.8 MiB [] 8% 3.3s162.8 MiB [] 9% 3.3s162.8 MiB [] 10% 3.2s162.8 MiB [] 11% 3.2s162.8 MiB [] 11% 3.1s162.8 MiB [] 12% 3.1s162.8 MiB [] 13% 3.1s162.8 MiB [] 14% 3.0s162.8 MiB [] 15% 2.9s162.8 MiB [] 15% 3.0s162.8 MiB [] 16% 2.9s162.8 MiB [] 17% 2.8s162.8 MiB [] 18% 2.7s162.8 MiB [] 19% 2.6s162.8 MiB [] 20% 2.6s162.8 MiB [] 21% 2.5s162.8 MiB [] 22% 2.5s162.8 MiB [] 22% 2.4s162.8 MiB [] 23% 2.5s162.8 MiB [] 24% 2.4s162.8 MiB [] 25% 2.4s162.8 MiB [] 26% 2.3s162.8 MiB [] 27% 2.2s162.8 MiB [] 28% 2.1s162.8 MiB [] 29% 2.1s162.8 MiB [] 30% 2.0s162.8 MiB [] 31% 2.0s162.8 MiB [] 32% 2.0s162.8 MiB [] 32% 1.9s162.8 MiB [] 33% 1.9s162.8 Mi

In [9]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [10]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [13]:
for season in SEASONS:
    await scrape_season(season)

2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2009-10 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2010-11 NBA Schedule | Basketball-Reference.com
2011-12 NBA Schedule | Basketball-Refere

TypeError: write() argument must be str, not None